# Section 3: Advanced optimization

### Initial Values

To start solving any optimization problem, initial values are required.
To facilitate the definition of starting points, the `OptimizationProblem` provides a `create_initial_values` method.

```{note}
This method only works if all optimization variables have defined lower and upper bounds.

Moreover, this method only guarantees that linear constraints are fulfilled.
Any nonlinear constraints may not be satisfied by the generated samples, and nonlinear parameter dependencies can be challenging to incorporate.
```

By default, the method returns a random point from the feasible region of the parameter space.
For this purpose, [hopsy](https://modsim.github.io/hopsy/) is used to efficiently (uniformly) sample the parameter space.
To create initial values, call `create_initial_values` and specify the number of individuals that should be returned.

In [ ]:
x0 = rosenbrock_problem.create_initial_values(10)
print(x0)

Alternatively, the Chebyshev center of the polytope can be computed, which is the center of the largest Euclidean ball that is fully contained within that polytope.

In [ ]:
x0 = rosenbrock_problem.get_chebyshev_center()
print(x0)

Let's create a method to visualize these points in the parameter space.

In [ ]:
def plot_initial_values(x0):
    import matplotlib.pyplot as plt
    import numpy as np
    fig, ax = plt.subplots()
    try:
        ax.scatter(x0[:, 0], x0[:, 1])
        ax.set_xlabel(r'$x_0$')
        ax.set_ylabel(r'$x_1$')
    except IndexError:
        ax.scatter(x0, np.ones_like((x0)))
        ax.set_xlabel(r'$x_0$')
    fig.tight_layout()

x0 = rosenbrock_problem.create_initial_values(500)
plot_initial_values(x0)

## Parameter Normalization
Most optimization algorithms struggle when optimization variables spread over multiple orders of magnitude.
**CADET-Process** provides several transformation methods which can help to soften these challenges.

```{figure} ./figures/transform.png
```

### Linear Normalization
The linear normalization maps the variable space from the lower and upper bound to a range between $0$ and $1$ by applying the following transformation:

$$
x^\prime = \frac{x - x_{lb}}{x_{ub} - x_{lb}}
$$

### Log Normalization
The log normalization maps the variable space from the lower and upper bound to a range between $0$ and $1$ by applying the following transformation:

$$
x^\prime = \frac{log \left( \frac{x}{x_{lb}} \right) }{log \left( \frac{x_{ub} }{x_{lb}} \right) }
$$

### Auto Transform
This transform will automatically switch between a linear and a log transform if the ratio of upper and lower bounds is larger than some value ($1000$ by default).

```{figure} ./figures/evaluation_example.png
```

## Evaluation Objects

```{figure} ./figures/evaluation_steps.png
```

- `OptimizationVariables` usually refers to attributes of a `Process` model (e.g. model parameters / event times.
- `EvaluationObject` objects manage the value of that optimization variable
- `Evaluators` execute (intermediate) steps required for calculating the objective (e.g. simulation)

```{figure} ./figures/evaluation_single_variable.png
:width: 30%
```


To associate an `OptimizationVariable` with an `EvaluationObject`, it first needs to be added to the `OptimizationProblem`.
For this purpose, consider a simple `Process` object from the [examples collection](https://cadet-process.readthedocs.io/en/stable/examples/batch_elution/process.html).

Then add the variable. In addition, specify:

- `parameter_path`: Path to the variable in the evaluation object
- `evaluation_objects`: The evaluation object(s) for which the variable should be set.

## Multiple Evaluation Objects

```{figure} ./figures/evaluation_multiple_variables.png
:width: 30%
```

### Evaluators
Any callable function can be added as `Evaluator`, assuming the first argument is the result of the previous step and it returns a single result object which is then processed by the next step.

```{figure} ./figures/evaluation_steps.png
```

- Any callable function can be added as `Evaluator`.
- Each `Evaluator` takes the previous result as input and returns a new (intermediate) result.
- Intermediate results are automatically cached.

## Evaluator Example

In this example, two steps are required:
- Process Simulation
- Fractionation

## Adding Objectives

Now, when adding objectives, specify which steps are required for each objective

## Evaluate Toolchain

To check the toolchain, simply call `evaluate_objectives`

In [ ]:
optimization_problem.objective_labels